In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer ,ENGLISH_STOP_WORDS

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import re 

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, LSTM, Embedding, Bidirectional
from sklearn.preprocessing import LabelEncoder


In [2]:
train = pd.read_csv('../input/emotion-data/train.csv')

train.shape

In [3]:
test = pd.read_csv('../input/emotion-data/test.csv')
test.head()

In [4]:
val= pd.read_csv('../input/emotion-data/val.csv')
val.head()

# Prepare data for model 


In [6]:
def clean_html(text):
    
    clean = re.compile('<.*?>')
    return re.sub(clean, '',text)
    
train['text']=train['text'].apply(clean_html)
test['text']=test['text'].apply(clean_html)
val['text']=val['text'].apply(clean_html)

train.head()

In [7]:
def convert_lower(text):
    return text.lower()

train['text']=train['text'].apply(convert_lower)
test['text']=test['text'].apply(convert_lower)
val['text']=val['text'].apply(convert_lower)
train.head()

In [8]:
def cleaning_tags(text):
    return ' '.join(re.sub("([@#][A-Za-z0-9_]+)|(\w+:\/\/\S+)"," ", text).split())
train['text']=train['text'].apply(cleaning_tags)
test['text']=test['text'].apply(cleaning_tags)
val['text']=val['text'].apply(cleaning_tags)
train.head()

In [10]:
import string
english_punctuations = string.punctuation
punctuations_list = english_punctuations
def cleaning_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)
train['text']=train['text'].apply(cleaning_punctuations)
test['text']=test['text'].apply(cleaning_punctuations)
val['text']=val['text'].apply(cleaning_punctuations)
train.head()

In [ ]:
def cleaning_repeating_char(text):
    return re.sub(r'([a-z])\1+', r'\1', text)
train['text']=train['text'].apply(cleaning_repeating_char)
test['text']=test['text'].apply(cleaning_repeating_char)
val['text']=val['text'].apply(cleaning_repeating_char)
train.head()

In [11]:
def cleaning_URLs(data):
    return re.sub('((www.[^s]+)|(https?://[^s]+))',' ',data)
train['text']=train['text'].apply(cleaning_URLs)
test['text']=test['text'].apply(cleaning_URLs)
val['text']=val['text'].apply(cleaning_URLs)
train.head()

In [12]:
def cleaning_numbers(data):
    return re.sub('[0-9]+', '', data)
train['text']=train['text'].apply(cleaning_numbers)
test['text']=test['text'].apply(cleaning_numbers)
val['text']=val['text'].apply(cleaning_numbers)
train.head()

In [13]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
train['text'] = train['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
test['text'] = test['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

val['text'] = val['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

train.head()

In [14]:
from nltk.stem.porter import PorterStemmer
ps= PorterStemmer()
y=[]

def stem_words(text):
    for i in text:
        y.append(ps.stem(i))
    z=y[:]
    y.clear()
    return z
train['text']=train['text'].apply(stem_words)
test['text']=test['text'].apply(stem_words)
val['text']=val['text'].apply(stem_words)
train.head()

In [15]:
def joinback2(list_input):
    return "".join(list_input)
    


train['text']=train['text'].apply(joinback2)
test['text']=test['text'].apply(joinback2)
val['text']=val['text'].apply(joinback2)
train.head()

In [16]:
vocab_size = 10000

In [17]:
train["length"] = [len(i) for i in train["text"]]


In [18]:
train["length"].max()
len_sentence = 150


In [19]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
def text_prepare(data,coulmn) :
    one_hot_word = [one_hot(word, n=vocab_size) for word in data[coulmn]]
    embeddec_doc = pad_sequences(sequences=one_hot_word,
                              maxlen=len_sentence,
                              padding="pre")
    print(data.shape)
    return embeddec_doc

In [20]:
x_train=text_prepare(train, "text")
x_validate=text_prepare(test, "text")
x_test=text_prepare(val, "text")

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(pd.concat([x_train, x_test], axis=0))


sequences_train = tokenizer.texts_to_sequences(x_train)
sequences_test = tokenizer.texts_to_sequences(x_test)
sequences_val = tokenizer.texts_to_sequences(x_validate)

x_train = pad_sequences(sequences_train, maxlen=256, truncating='pre')
x_test = pad_sequences(sequences_test, maxlen=256, truncating='pre')
x_validate = pad_sequences(sequences_val, maxlen=256, truncating='pre')

vocabSize = len(tokenizer.index_word) + 1
print(f"Vocabulary size = {vocabSize}")

In [21]:
y_train=train["label"]
y_validate=test["label"]
y_test=val["label"]

In [23]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
y_train = np.array(y_train)
y_train = enc.fit_transform(y_train.reshape(-1,1)).toarray()
y_test = np.array(y_test)
y_validate = np.array(y_validate)

y_test = enc.fit_transform(y_test.reshape(-1,1)).toarray()
y_validate = enc.fit_transform(y_validate.reshape(-1,1)).toarray()

In [24]:
y_train.shape

In [25]:
x_train.shape

In [26]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam

In [27]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=150, input_length=len_sentence))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(64, activation="sigmoid"))
model.add(Dropout(0.2))
model.add(Dense(7, activation="softmax"))

In [28]:
model.compile(optimizer="Adam", loss = "categorical_crossentropy", metrics=["accuracy"])


In [30]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 5)
mc = ModelCheckpoint('./model.h5', monitor = 'val_accuracy', mode = 'max', verbose = 1, save_best_only = True)

In [31]:
hist = model.fit(x_train, y_train, epochs = 25, batch_size = 64, validation_data=(x_validate, y_validate),verbose = 1, callbacks= [es, mc])

In [33]:
model.save_weights("model.h5")
print("Saved model to disk")


In [ ]:
le =LabelEncoder()

In [ ]:
tokenizer=Tokenizer()
sentences = [
            "Happy",
            "Your point on this certain matter made me outrageous, how can you say so? This is insane.",
            "I can't do it, I'm not ready to lose anything, just leave me alone",
            "Merlin's beard harry, you can cast the Patronus charm! I'm amazed!"
            ]
for tf in sentences:
    tf=clean_html(tf)
    tf=convert_lower(tf)
    tf=cleaning_tags(tf)
    tf=cleaning_punctuations(tf)
    tf=cleaning_repeating_char(tf)
    tf=cleaning_URLs(tf)
    tf=stem_words(tf)
    tf=joinback2(tf)
    tf=clean(tf)
    print(tf)
   
    print(sentence)
    sentence = pad_sequences(sentence, maxlen=256, truncating='pre')
    result = (np.argmax(model.predict(sentence), axis=-1))
    proba =  np.max(model.predict(sentence))
    print(f"{result} : {proba}\n\n")
    


In [ ]:
adam = Adam(learning_rate=0.005)

model = Sequential()
model.add(Embedding(vocabSize, 200, input_length=X_train.shape[1], weights=[embedding_matrix], trainable=False))
model.add(Bidirectional(LSTM(256, dropout=0.2,recurrent_dropout=0.2, return_sequences=True)))
model.add(Bidirectional(LSTM(128, dropout=0.2,recurrent_dropout=0.2, return_sequences=True)))
model.add(Bidirectional(LSTM(128, dropout=0.2,recurrent_dropout=0.2)))
model.add(Dense(6, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
model.summary()


In [ ]:
adam = Adam(learning_rate=0.005)

model = Sequential()
model.add(Embedding(10000, 200, input_length=x_train.shape[1], trainable=False))
model.add(Bidirectional(LSTM(256, dropout=0.2,recurrent_dropout=0.2, return_sequences=True)))
model.add(Bidirectional(LSTM(128, dropout=0.2,recurrent_dropout=0.2, return_sequences=True)))
model.add(Bidirectional(LSTM(128, dropout=0.2,recurrent_dropout=0.2)))
model.add(Dense(7, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
model.summary()

In [ ]:
callback = EarlyStopping(
    monitor="val_loss",
    patience=2,
    restore_best_weights=True,
)

In [ ]:
hist2 = model.fit(x_train, y_train, epochs = 25, batch_size = 64, validation_data=(x_validate, y_validate),verbose = 1, callbacks= [es, mc])